# 8 Deep Learning with Keras

## 8.2 Convolutional Neural Networks with Keras

### MNIST Dataset

MNIST dataset is a popular benchmarking dataset in which we ave samples of hand-written symbols for 0-9 digits. Here are the properties:
- 60k training images, 10k test images
- Each image is 28x28 pixels, single channel
- Pixel values are normalized to [0,1]
- The dataset is a 4-dimensional array:
    (Samples, Width, Height, Channels) -> (60000, 28, 28, 1)
- Labels come in an array in which each element is the digit label 0-9; but we transform it into the **one-hot-encondig**: 0/1 for each of the 0-9 classes, i.e.: (Samples, Classes) -> (60000, 10); 4 = (0,0,0,0,1,0,0,0,0,0)